This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_conversion/xgboost-autoconversion).

## Introduction

The following tutorial is a brief example of how to convert a [XGBoost](https://xgboost.readthedocs.io/en/stable/index.html) Classification ML model with the `convert_model` method and upload it into your Wallaroo instance.

This tutorial assumes that you have a Wallaroo instance and are running this Notebook from the Wallaroo Jupyter Hub service.

* Convert a `XGBoost` Classification ML model and upload it into the Wallaroo engine.
* Run a sample inference on the converted model in a Wallaroo instance.

This tutorial provides the following:

* `xgb_class.pickle`: A pretrained `XGBoost` Classification model with 25 columns.
* `xgb_class_eval.json`: Test data to perform a sample inference.

## Prerequisites

Wallaroo supports the following model versions:

* XGBoost:  Version 1.6.0
* SKLearn: 1.1.2

## Conversion Steps

To use the Wallaroo autoconverter `convert_model(path, source_type, conversion_arguments)` method takes 3 parameters.  The parameters for `XGBoost` conversions are:

* `path` (STRING):  The path to the ML model file.
* `source_type` (ModelConversionSource): The type of ML model to be converted.  As of this time Wallaroo auto-conversion supports the following source types and their associated `ModelConversionSource`:
  * **sklearn**: `ModelConversionSource.SKLEARN`
  * **xgboost**: `ModelConversionSource.XGBOOST`
  * **keras**: `ModelConversionSource.KERAS`
* `conversion_arguments`:  The arguments for the conversion based on the type of model being converted.  These are:
    * `wallaroo.ModelConversion.ConvertXGBoostArgs`: Used for `XGBoost` models and takes the following parameters:
    * `name`: The name of the model being converted.
    * `comment`: Any comments for the model.
    * `number_of_columns`: The number of columns the model was trained for.
    * `input_type`: A [tensorflow Dtype](https://www.tensorflow.org/api_docs/python/tf/dtypes/DType) called in the format `ModelConversionInputType.{type}`, where `{type}` is `Float`, `Double`, etc depending on the model.

### Import Libraries

The first step is to import the libraries needed.

In [1]:
import wallaroo

from wallaroo.ModelConversion import ConvertXGBoostArgs, ModelConversionSource, ModelConversionInputType
from wallaroo.object import EntityNotFoundError

import os
# For Wallaroo SDK 2023.1
os.environ["ARROW_ENABLED"]="True"

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Connect to Wallaroo

Connect to your Wallaroo instance and store the connection into the variable `wl`.

In [2]:
# Login through local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR PREFIX"

wallarooPrefix = "doc-test"
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

Please log into the following URL in a web browser:

	https://doc-test.keycloak.wallaroocommunity.ninja/auth/realms/master/device?user_code=BRVK-TCJK

Login successful!


### Configuration and Methods

The following will set the workspace, pipeline, model name, the model file name used when uploading and converting the `keras` model, and the sample data.

The functions `get_workspace(name)` will either set the current workspace to the requested name, or create it if it does not exist.  The function `get_pipeline(name)` will either set the pipeline used to the name requested, or create it in the current workspace if it does not exist.

In [3]:
workspace_name = 'xgboost-classification-autoconvert-workspace'
pipeline_name = 'xgboost-classification-autoconvert-pipeline'
model_name = 'xgb-class-model'
model_file_name = 'xgb_class.pickle'

def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

### Set the Workspace and Pipeline

Set or create the workspace and pipeline based on the names configured earlier.

In [4]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
pipeline

name,xgboost-classification-autoconvert-pipeline
created,2023-04-05 18:13:49.170738+00:00
last_updated,2023-04-05 18:13:49.170738+00:00
deployed,(none)
tags,
versions,ba41772e-8c8e-4b7a-8373-98e6096c6480
steps,


### Set the Model Autoconvert Parameters

Set the paramters for converting the `xgb-class-model`.

In [5]:
#the number of columns
NF = 25

model_conversion_args = ConvertXGBoostArgs(
    name=model_name,
    comment="xgboost classification model test",
    number_of_columns=NF,
    input_type=ModelConversionInputType.Float32
)
model_conversion_type = ModelConversionSource.XGBOOST

### Upload and Convert the Model

Now we can upload the convert the model.  Once finished, it will be stored as `{unique-file-id}-converted.onnx`.

In [6]:
# convert and upload
model_wl = wl.convert_model(model_file_name, model_conversion_type, model_conversion_args)

## Test Inference

With the model uploaded and converted, we can run a sample inference.

### Deploy the Pipeline

Add the uploaded and converted `model_wl` as a step in the pipeline, then deploy it.

In [7]:
pipeline.add_model_step(model_wl).deploy()

name,xgboost-classification-autoconvert-pipeline
created,2023-04-05 18:13:49.170738+00:00
last_updated,2023-04-05 18:13:52.566924+00:00
deployed,True
tags,
versions,"9cf6b8a1-3c3e-47de-a314-82e11c89f689, ba41772e-8c8e-4b7a-8373-98e6096c6480"
steps,xgb-class-model


### Run the Inference

Use the evaluation data to verify the process completed successfully.

In [8]:
sample_data = 'xgb_class_eval.df.json'
result = pipeline.infer_from_file(sample_data)
display(result)

,time,in.tensor,out.label,out.probabilities,check_failures
0,2023-04-05 18:14:04.304,"[-0.9650039837, 1.7162569382, 1.8570196174, 0.7225873636, 1.4614264692, 1.9567455469, 3.1280554236, 2.4737274835, 2.045634687, 0.0697759683, -0.7334890238, 1.4661397464, -1.7339080123, -0.3295498275, -0.5405674404, 0.9325072938, -0.1753815275, 0.8389569878, 0.2995238298, 2.020354449, 0.307715435, -0.786562628, 1.6198295619, -3.1550540615, 2.4493095715]",[0],"[0.8773107, 0.12268931]",0
1,2023-04-05 18:14:04.304,"[-0.24290676, -2.7621478465, -1.0460044448, -0.4367771067, 0.7114974086, 3.1152360132, 0.8780655791, 1.5959052391, 0.1291853603, -0.4705432269, -0.2870965835, 0.2758634598, -2.5296629025, -0.8581708475, -0.0447250952, -0.8147113092, 0.3394927614, 0.1165005518, 0.5214230106, 1.0323965467, 0.824008803, -0.2602068525, -2.5164397098, -2.2480625668, 0.7147467132]",[0],"[0.9160531, 0.08394688]",0
2,2023-04-05 18:14:04.304,"[0.3261925153, -1.1340263025, -0.0210165684, -0.402436985, 0.1136841647, 1.9756910921, -1.6567823116, -3.0377564302, 1.0839562248, 1.535350752, -1.5641493986, -0.4037836272, -0.0502258358, -1.383033319, -2.1692714889, 0.5474654104, 0.5884733316, -0.6575750129, -0.4456088906, 1.9450809267, -0.5395060067, 0.0020371202, -2.0035740797, 5.3368805176, -1.3683109303]",[1],"[0.01365149, 0.9863485]",0
3,2023-04-05 18:14:04.304,"[0.7071268106, -1.1177500788, 0.1311702635, -0.0342823916, 1.4166474292, -0.7600812269, -1.643252821, 1.1809622308, 1.1552655664, -1.4616319423, -1.3196760448, -0.3871231717, -1.0052010294, 0.3757483273, 0.8164121104, 0.6636194102, 0.2054206669, 0.3971757239, 1.0712736575, 0.5687901164, 0.545534547, -0.4022272078, 0.5202183853, -1.1450692638, -1.6687803276]",[0],"[0.8232141, 0.17678589]",0
4,2023-04-05 18:14:04.304,"[-0.4753271684, 0.9648567582, 4.1002801029, -0.3474129796, 0.5912316716, -0.3616544697, -2.9339075495, 0.8583809009, -0.7625328481, -1.447786717, -0.0183969915, -0.1028844583, -1.9931308252, -0.6141588978, 1.5368353642, -0.5482829279, 2.1576770706, 0.4772412627, 0.9956210462, 1.7124754134, -0.7415852899, -0.3876944367, 5.7178008466, 7.1237030134, 0.1815704771]",[0],"[0.5882849, 0.4117151]",0


### Undeploy the Pipeline

With the tests complete, we will undeploy the pipeline to return the resources back to the Wallaroo instance.

In [10]:
pipeline.undeploy()

name,xgboost-classification-autoconvert-pipeline
created,2023-04-05 18:13:49.170738+00:00
last_updated,2023-04-05 18:13:52.566924+00:00
deployed,False
tags,
versions,"9cf6b8a1-3c3e-47de-a314-82e11c89f689, ba41772e-8c8e-4b7a-8373-98e6096c6480"
steps,xgb-class-model
